In [77]:
from brickschema import Graph
from rdflib import Namespace
import os

In [78]:

class BasicValidationInterface:

    def __init__(self, graph_path: str, nightly: bool = False):
        self.graph_path = graph_path
        if nightly:
            self.graph = Graph(load_brick_nightly=True)
        else:
            self.graph = Graph(load_brick=True)
        # loading graph in init
        self.graph.load_file(self.graph_path)

    def validate(self):
        valid, _, report = self.graph.validate()
        print(f"Graph <{self.graph_path}> is valid? {valid}")
        if not valid:
            print("-" * 79)
            print(report)
            print("-" * 79)


In [79]:
deprecation_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a ?class}
INSERT {?this a ?newClass}
WHERE {
?this a ?class .
?class owl:deprecated true .
?class brick:deprecatedInVersion ?depver .
?class brick:isReplacedBy ?newClass .
}
"""

In [80]:
#
ft2_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:hasUnits unit:FT_2}
INSERT {?this brick:hasUnit unit:FT2}
WHERE {
?this brick:hasUnits unit:FT_2
}
"""

In [81]:
bldg1_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 9973 }
WHERE {
?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""

In [82]:
bv = BasicValidationInterface(graph_path='graphs/bldg1.ttl', nightly=True)
bv.graph.update(deprecation_fix)
bv.graph.update(bldg1_area_fix)
bv.graph.update(ft2_fix)

In [84]:
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

<Graph identifier=N4adb8391541442bea5d9403b2cdd9d02 (<class 'brickschema.graph.Graph'>)>

In [85]:
bv.validate()

Graph <graphs/bldg1.ttl> is valid? True
